In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
with open('pizza_request_dataset.json') as f:
  data = json.load(f)

In [5]:
df = pd.read_json('pizza_request_dataset.json')
train, test = train_test_split(df, test_size=567, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
word_vectorizer = CountVectorizer(ngram_range=(1,1), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(train['request_text'])
frequencies = sum(sparse_matrix).toarray()[0]
train_ngrams = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])

In [ ]:
topp_500_unigrams = train_ngrams.sort_values("frequency", ascending=False).head(500)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
word_vectorizer = CountVectorizer(ngram_range=(2,2), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(train['request_text'])
frequencies = sum(sparse_matrix).toarray()[0]
train_bigrams = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])

In [44]:
topp500_bigrams = train_bigrams.sort_values("frequency", ascending=False).head(500)

In [45]:
topp500_bigrams['bigrams'] = topp500_bigrams.index

In [46]:
topp_500_unigrams['unigrams'] = topp_500_unigrams.index
unigrams = topp_500_unigrams['unigrams'].tolist()
bigrams =topp500_bigrams['bigrams'].tolist()
unigrams
ngrams = unigrams + bigrams
#ngrams

In [47]:
#create X to be 5104x1000 shape
X = []
for text in train['request_text'].tolist():
  contains_ngram =[]
  for i in range(len(ngrams)):
    if ngrams[i] in text:
      contains_ngram.append(1)
    else:
      contains_ngram.append(0)
  X.append(contains_ngram)

In [48]:
X = np.array(X)
X.shape

(5104, 1000)

In [6]:
y = train['requester_received_pizza']

In [7]:
y_int = [int(val == True) for val in y]
len(y_int)

5104

In [51]:
from sklearn import svm
from sklearn import linear_model
clf = svm.SVC(kernel='linear')
clf.fit(X, y_int)
#svm = sklearn.linear_model.SGDClassifier(loss='hinge')

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [52]:
#sklearn.linear_model.SGDClassifier¶X_test = []
X_test = []
for text in test['request_text'].tolist():
  contains_ngram =[]
  for i in range(len(ngrams)):
    if ngrams[i] in text:
      contains_ngram.append(1)
    else:
      contains_ngram.append(0)
  X_test.append(contains_ngram)
y_test = test['requester_received_pizza']
y_test = [int(val == True) for val in y_test]


In [53]:
test_set_preds = clf.predict(X_test)

In [54]:
import sklearn.metrics

accuracy_1a = clf.score(X_test, y_test)
precision_1a = sklearn.metrics.precision_score(y_test, test_set_preds)
recall_1a = sklearn.metrics.recall_score(y_test, test_set_preds)
f1_1a = sklearn.metrics.f1_score(y_test, test_set_preds)
auc_1a= sklearn.metrics.roc_auc_score(y_test, test_set_preds)

tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, test_set_preds).ravel()
specificity_1a = tn / (tn+fp)

print("Answers for 1a: ")
print("accuracy: " + str(accuracy_1a))
print("precision: " +str(precision_1a))
print("recall: " + str(recall_1a))
print("f1: " + str(f1_1a))
print("auc: " + str(auc_1a))
print("specificity: " + str(specificity_1a))

Answers for 1a: 
accuracy: 0.7072310405643739
precision: 0.3924050632911392
recall: 0.2080536912751678
f1: 0.2719298245614035
auc: 0.5466105776950002
specificity: 0.8851674641148325


Problem 1b

In [12]:
feature_cols = ['post_was_edited','requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', #,
                'requester_days_since_first_post_on_raop_at_request','requester_days_since_first_post_on_raop_at_retrieval',
                'requester_number_of_comments_at_request','requester_number_of_comments_at_retrieval','requester_number_of_comments_in_raop_at_request',
                'requester_number_of_comments_in_raop_at_retrieval','requester_number_of_posts_at_request','requester_number_of_posts_at_retrieval',
                'requester_number_of_posts_on_raop_at_request','requester_number_of_posts_on_raop_at_retrieval','requester_number_of_subreddits_at_request',
                'requester_subreddits_at_request', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval',
                'requester_upvotes_minus_downvotes_at_request', 'requester_upvotes_minus_downvotes_at_retrieval', 'requester_upvotes_plus_downvotes_at_request',
                'requester_upvotes_plus_downvotes_at_retrieval', 'requester_user_flair']
              

In [13]:
X_train_1b = train.loc[:, feature_cols]
X_test_1b = test.loc[:, feature_cols]

In [14]:
subreddits = X_train_1b['requester_subreddits_at_request']
from itertools import chain, groupby
from collections import Counter
ll = subreddits.tolist()
top_subs = sorted(chain(*(Counter(l).items() for l in ll)), reverse=True)

In [15]:
Counter(chain.from_iterable(ll)).most_common(10)

[('AskReddit', 2904),
 ('pics', 2454),
 ('funny', 2412),
 ('IAmA', 1914),
 ('WTF', 1913),
 ('gaming', 1867),
 ('Random_Acts_Of_Pizza', 1759),
 ('videos', 1455),
 ('todayilearned', 1400),
 ('AdviceAnimals', 1308)]

Most Common Subreddits : ('AskReddit', 2904),
 ('pics', 2454),
 ('funny', 2412),
 ('IAmA', 1914),
 ('WTF', 1913),
 ('gaming', 1867),
 ('Random_Acts_Of_Pizza', 1759),
 ('videos', 1455),
 ('todayilearned', 1400),
 ('AdviceAnimals', 1308)]

In [ ]:
most_common_subs = ['AskReddit', 'pics', 'funny', 'IAmA', "WTF", 'gaming', 'Random_Acts_Of_Pizza', 'videos', 'todayilearned', 'AdviceAnimals']
sub_reddit_feats_train = []
for text in train['requester_subreddits_at_request'].tolist():
  print(text)
  contains_sub =[]
  for i in range(len(most_common_subs)):
    if most_common_subs[i] in text:
      contains_sub.append(1)
    else:
      contains_sub.append(0)
  sub_reddit_feats_train.append(contains_sub)
sub_reddit_feats_train

In [17]:
np.array(sub_reddit_feats_train).sum()

19386

In [18]:
np_subreddits_train = np.array(sub_reddit_feats_train)
np_subreddits_train.shape

(5104, 10)

In [19]:
most_common_subs = ['AskReddit', 'pics', 'funny', 'IAmA', "WTF", 'gaming', 'Random_Acts_Of_Pizza', 'videos', 'todayilearned', 'AdviceAnimals']
sub_reddit_feats_test = []
for text in test['requester_subreddits_at_request'].tolist():
  contains_sub =[]
  for i in range(len(most_common_subs)):
    if most_common_subs[i] in text:
      contains_sub.append(1)
    else:
      contains_sub.append(0)
  sub_reddit_feats_test.append(contains_sub)
np_subreddits_test = np.array(sub_reddit_feats_test)
np_subreddits_test.shape

(567, 10)

In [20]:
np_subreddits_test

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 0, ..., 0, 0, 1]])

In [21]:
#X_train_1b.drop(['F','I','P','h','m','o','r','s'], axis=1, inplace=True)
one_hot_flair = pd.get_dummies(X_train_1b['requester_user_flair'])
#one_hot_flair

In [22]:
X_train_1b_final = pd.concat([X_train_1b,one_hot_flair], axis=1)
X_train_1b_final = X_train_1b_final.drop(['requester_user_flair','requester_subreddits_at_request'],1)
#X_train_1b_final



In [69]:
X_train_1b_final

,post_was_edited,requester_account_age_in_days_at_request,requester_account_age_in_days_at_retrieval,requester_days_since_first_post_on_raop_at_request,requester_days_since_first_post_on_raop_at_retrieval,requester_number_of_comments_at_request,requester_number_of_comments_at_retrieval,requester_number_of_comments_in_raop_at_request,requester_number_of_comments_in_raop_at_retrieval,requester_number_of_posts_at_request,requester_number_of_posts_at_retrieval,requester_number_of_posts_on_raop_at_request,requester_number_of_posts_on_raop_at_retrieval,requester_number_of_subreddits_at_request,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,PIF,shroom
4023,0,23.688519,717.008947,0.000000,693.320428,1,1,0,0,0,1,0,1,2,2,3,19,19,31,31,0,0
2988,0,511.161817,605.643264,0.000000,94.439780,660,789,0,0,42,56,0,1,29,1,2,2565,2826,3807,4266,0,0
3313,1,60.862581,782.167894,0.000000,721.305313,0,999,0,0,12,157,0,1,8,2,5,237,4965,423,6931,0,0
3409,0,82.156586,158.056840,0.000000,0.000000,12,13,0,0,2,2,0,0,4,9,23,201,202,269,270,0,0
676,0,387.013611,630.427824,0.000000,243.372546,255,306,0,0,85,110,0,1,46,2,3,4412,7963,8346,15411,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0,542.096296,752.213438,0.000000,210.075475,0,999,0,0,40,59,0,1,19,2,8,1335,8269,2393,14207,0,0
5191,0,732.137812,1017.506262,0.000000,285.368449,56,112,0,0,6,14,0,1,17,1,1,758,981,1294,1649,0,0
5226,0,239.057743,1057.696377,0.000000,818.596968,38,169,0,2,51,127,0,1,23,1,2,911,4710,1959,9852,0,0
5390,0,72.029942,289.573750,72.029942,289.573750,3,4,2,3,1,2,1,2,2,1,4,6,11,8,17,0,0


In [23]:
one_hot_flair = pd.get_dummies(X_test_1b['requester_user_flair'])
X_test_1b_final = pd.concat([X_test_1b,one_hot_flair], axis=1)
X_test_1b_final = X_test_1b_final.drop(['requester_user_flair','requester_subreddits_at_request'],1)
X_test_1b_final

,post_was_edited,requester_account_age_in_days_at_request,requester_account_age_in_days_at_retrieval,requester_days_since_first_post_on_raop_at_request,requester_days_since_first_post_on_raop_at_retrieval,requester_number_of_comments_at_request,requester_number_of_comments_at_retrieval,requester_number_of_comments_in_raop_at_request,requester_number_of_comments_in_raop_at_retrieval,requester_number_of_posts_at_request,requester_number_of_posts_at_retrieval,requester_number_of_posts_on_raop_at_request,requester_number_of_posts_on_raop_at_retrieval,requester_number_of_subreddits_at_request,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,PIF,shroom
3671,0,0.000000,298.623287,0.000000,298.623287,0,5,0,1,0,1,0,1,1,14,17,2,-9,32,83,0,0
5183,0,205.548843,527.360718,0.000000,321.811875,108,131,0,0,24,28,0,1,25,4,4,574,633,956,1031,0,0
1859,0,411.171655,654.795544,0.000000,243.582222,413,622,0,1,56,80,0,4,53,4,5,2013,3280,3213,5564,0,1
2589,0,105.919167,227.387766,0.000000,121.426933,391,998,0,2,75,144,0,1,63,2,4,4209,7359,7143,13145,0,0
1662,0,362.890451,1275.413495,0.000000,912.481377,39,461,0,3,5,54,0,2,13,2,13,237,4837,303,7277,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,0,0.848299,330.163727,0.000000,329.315428,1,23,0,0,1,10,0,2,2,1,1,3,182,5,264,0,0
151,0,11.471829,800.084456,0.000000,788.570961,3,111,0,1,0,6,0,1,1,5,9,4,780,4,1224,0,1
763,0,500.034468,603.476435,95.836285,199.278252,499,513,0,2,25,27,1,2,48,4,5,6119,6163,11191,11277,0,0
3724,0,1297.815868,2091.248819,117.954213,911.387164,469,1000,3,5,29,114,0,1,32,0,2,749,1566,1113,2682,0,0


In [25]:
columns_not_to_normalize = ['post_was_edited', 'PIF', 'shroom', ]

In [81]:
columns =X_test_1b_final.columns.tolist()

In [86]:
len(columns)

22

In [26]:
def normalize(dataset, list_columns_keep):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    for col in list_columns_keep:
      dataNorm[col]=dataset[col]
    return dataNorm

In [27]:
X_train_norm = normalize(X_train_1b_final,columns_not_to_normalize  )
X_test_norm = normalize(X_test_1b_final, columns_not_to_normalize)
X_train_norm

,post_was_edited,requester_account_age_in_days_at_request,requester_account_age_in_days_at_retrieval,requester_days_since_first_post_on_raop_at_request,requester_days_since_first_post_on_raop_at_retrieval,requester_number_of_comments_at_request,requester_number_of_comments_at_retrieval,requester_number_of_comments_in_raop_at_request,requester_number_of_comments_in_raop_at_retrieval,requester_number_of_posts_at_request,requester_number_of_posts_at_retrieval,requester_number_of_posts_on_raop_at_request,requester_number_of_posts_on_raop_at_retrieval,requester_number_of_subreddits_at_request,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,PIF,shroom
4023,0,0.008431,0.237022,0.000000,0.676141,0.001004,0.001,0.000000,0.000000,0.000000,0.001001,0.0,0.111111,0.010753,0.020619,0.007246,0.001237,0.002097,0.000024,0.000015,0,0
2988,0,0.181924,0.197726,0.000000,0.092100,0.662651,0.789,0.000000,0.000000,0.048443,0.056056,0.0,0.111111,0.155914,0.010309,0.004831,0.017644,0.014619,0.002958,0.002085,0,0
3313,1,0.021661,0.260014,0.000000,0.703433,0.000000,0.999,0.000000,0.000000,0.013841,0.157157,0.0,0.111111,0.043011,0.020619,0.012077,0.002642,0.024161,0.000329,0.003387,0,0
3409,0,0.029240,0.039790,0.000000,0.000000,0.012048,0.013,0.000000,0.000000,0.002307,0.002002,0.0,0.000000,0.021505,0.092784,0.055556,0.002410,0.002913,0.000209,0.000132,0,0
676,0,0.137739,0.206471,0.000000,0.237342,0.256024,0.306,0.000000,0.000000,0.098039,0.110110,0.0,0.111111,0.247312,0.020619,0.007246,0.029546,0.037536,0.006486,0.007530,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0,0.192934,0.249444,0.000000,0.204870,0.000000,0.999,0.000000,0.000000,0.046136,0.059059,0.0,0.111111,0.102151,0.020619,0.019324,0.009718,0.038901,0.001860,0.006942,0,0
5191,0,0.260570,0.343056,0.000000,0.278298,0.056225,0.112,0.000000,0.000000,0.006920,0.014014,0.0,0.111111,0.091398,0.010309,0.002415,0.005999,0.006388,0.001006,0.000806,0,0
5226,0,0.085081,0.357237,0.000000,0.798314,0.038153,0.169,0.000000,0.014388,0.058824,0.127127,0.0,0.111111,0.123656,0.010309,0.004831,0.006985,0.023024,0.001522,0.004814,0,0
5390,0,0.025636,0.086197,0.091704,0.282399,0.003012,0.004,0.022727,0.021583,0.001153,0.002002,0.2,0.222222,0.010753,0.010309,0.009662,0.001153,0.002061,0.000006,0.000008,0,0


In [28]:
X_test_norm

,post_was_edited,requester_account_age_in_days_at_request,requester_account_age_in_days_at_retrieval,requester_days_since_first_post_on_raop_at_request,requester_days_since_first_post_on_raop_at_retrieval,requester_number_of_comments_at_request,requester_number_of_comments_at_retrieval,requester_number_of_comments_in_raop_at_request,requester_number_of_comments_in_raop_at_retrieval,requester_number_of_posts_at_request,requester_number_of_posts_at_retrieval,requester_number_of_posts_on_raop_at_request,requester_number_of_posts_on_raop_at_retrieval,requester_number_of_subreddits_at_request,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,PIF,shroom
3671,0,0.000000,0.076778,0.000000,0.322287,0.000000,0.005,0.000000,0.016129,0.000000,0.000000,0.0,0.166667,0.004255,0.411765,0.061594,0.000415,0.000000,0.000234,0.000389,0,0
5183,0,0.091958,0.163111,0.000000,0.347313,0.108871,0.131,0.000000,0.000000,0.054299,0.031765,0.0,0.166667,0.106383,0.117647,0.014493,0.022020,0.011542,0.007000,0.004887,0,0
1859,0,0.183949,0.211209,0.000000,0.262884,0.416331,0.622,0.000000,0.016129,0.126697,0.092941,0.0,0.666667,0.225532,0.117647,0.018116,0.076371,0.059128,0.023527,0.026393,0,1
2589,0,0.047386,0.049891,0.000000,0.131049,0.394153,0.998,0.000000,0.032258,0.169683,0.168235,0.0,0.166667,0.268085,0.058824,0.014493,0.159314,0.132458,0.052304,0.062359,0,0
1662,0,0.162349,0.445451,0.000000,0.984788,0.039315,0.461,0.000000,0.048387,0.011312,0.062353,0.0,0.333333,0.055319,0.058824,0.047101,0.009291,0.087119,0.002219,0.034520,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,0,0.000380,0.088683,0.000000,0.355411,0.001008,0.023,0.000000,0.000000,0.002262,0.010588,0.0,0.333333,0.008511,0.029412,0.003623,0.000453,0.003434,0.000037,0.001248,0,0
151,0,0.005132,0.266046,0.000000,0.851058,0.003024,0.111,0.000000,0.016129,0.000000,0.005882,0.0,0.166667,0.004255,0.147059,0.032609,0.000491,0.014184,0.000029,0.005802,0,1
763,0,0.223704,0.191840,0.191731,0.215069,0.503024,0.513,0.000000,0.032258,0.056561,0.030588,0.2,0.333333,0.204255,0.117647,0.018116,0.231455,0.110957,0.081945,0.053497,0,0
3724,0,0.580614,0.753375,0.235980,0.983607,0.472782,1.000,0.053571,0.080645,0.065611,0.132941,0.0,0.166667,0.136170,0.000000,0.007246,0.028630,0.028315,0.008150,0.012719,0,0


In [29]:
X_train_np = X_train_norm.to_numpy()
X_test_np = X_test_norm.to_numpy()
X_train_np.shape

(5104, 22)

In [30]:
X_test_np.shape

(567, 22)

In [31]:
final_X_train_np = np.concatenate((X_train_np,np_subreddits_train),axis=1)
final_X_test_np = np.concatenate((X_test_np,np_subreddits_test),axis=1)
final_X_train_np

array([[0.        , 0.00843083, 0.23702223, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.18192425, 0.19772573, ..., 1.        , 0.        ,
        1.        ],
       [1.        , 0.0216612 , 0.26001422, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.08508148, 0.35723721, ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.0256357 , 0.08619742, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04827613, 0.22009133, ..., 0.        , 0.        ,
        0.        ]])

In [34]:
final_X_train_np.shape

(5104, 32)

In [32]:
#normalize
final_X_test_np

array([[0.        , 0.        , 0.07677809, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.09195805, 0.16311123, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.18394918, 0.21120938, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.        , 0.22370446, 0.19183984, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.58061437, 0.75337477, ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.09055006, 0.20301815, ..., 0.        , 0.        ,
        1.        ]])

In [35]:
final_X_test_np.shape

(567, 32)

In [ ]:
# clf = svm.SVC(kernel='linear')
# clf.fit(final_X_train_np, y_int)

In [37]:
from sklearn.linear_model import SGDClassifier

In [38]:
clf = SGDClassifier(loss="hinge").fit(final_X_train_np, y_int)

In [39]:
#combine these three dataframes: X_test_1b_no_flair, sub_one_hot_test_df, test_one_hot_flair, 
test_set_preds = clf.predict(final_X_test_np)

In [42]:
y_test = test['requester_received_pizza']
y_test = [int(val == True) for val in y_test]


In [43]:
import sklearn.metrics

accuracy_1a = clf.score(final_X_test_np, y_test)
precision_1a = sklearn.metrics.precision_score(y_test, test_set_preds)
recall_1a = sklearn.metrics.recall_score(y_test, test_set_preds)
f1_1a = sklearn.metrics.f1_score(y_test, test_set_preds)
auc_1a= sklearn.metrics.roc_auc_score(y_test, test_set_preds)

tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, test_set_preds).ravel()
specificity_1a = tn / (tn+fp)


In [ ]:
#clf = SGDClassifier(loss="hinge").fit(X_train_1b_lst, y_int)
import sklearn.metrics

accuracy_1a = clf.score(normed_matrix_test, y_test)
precision_1a = sklearn.metrics.precision_score(y_test, test_set_preds)
recall_1a = sklearn.metrics.recall_score(y_test, test_set_preds)
f1_1a = sklearn.metrics.f1_score(y_test, test_set_preds)
auc_1a= sklearn.metrics.roc_auc_score(y_test, test_set_preds)

tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, test_set_preds).ravel()
specificity_1a = tn / (tn+fp)

In [44]:
print("Answers for 1b: ")
print("accuracy: " + str(accuracy_1a))
print("precision: " +str(precision_1a))
print("recall: " + str(recall_1a))
print("f1: " + str(f1_1a))
print("auc: " + str(auc_1a))
print("specificity " + str(specificity_1a))

Answers for 1b: 
accuracy: 0.9153439153439153
precision: 0.9391304347826087
recall: 0.7248322147651006
f1: 0.8181818181818182
auc: 0.8540429016409234
specificity 0.9832535885167464


In [ ]:
y_test = test['requester_received_pizza']
y_test = [int(val == True) for val in y_test]

Model 3

In [ ]:
desire_set  = set(line.strip() for line in open('desire.txt'))
family_set = set(line.strip() for line in open('family.txt'))
job_set = set(line.strip() for line in open('job.txt'))
money_set = set(line.strip() for line in open('money.txt'))
student_set = set(line.strip() for line in open('student.txt'))

In [ ]:
all_sets = [desire_set, family_set, job_set, money_set, student_set]

In [ ]:
X_train_1c = []
for text in train['request_text'].tolist():
  total_word_count = len(text.split())

  row =[]
  for i in range(len(all_sets)):
    curr_set = all_sets[i]
    set_word_count = 0
    for word in curr_set:
      if word in text:
        set_word_count +=1
    if total_word_count > 0:
      row.append(set_word_count/total_word_count)
    else:
      row.append(0)
  X_train_1c.append(row)

X_train_1c

In [ ]:
X_train_1c_np = np.array(X_train_1c)
X_train_1c_np.shape

(5104, 5)

In [ ]:
X_test_1c = []
for text in test['request_text'].tolist():
  total_word_count = len(text.split())

  row =[]
  for i in range(len(all_sets)):
    curr_set = all_sets[i]
    set_word_count = 0
    for word in curr_set:
      if word in text:
        set_word_count +=1
    if total_word_count > 0:
      row.append(set_word_count/total_word_count)
    else:
      row.append(0)
  X_test_1c.append(row)

X_test_1c
X_test_1c_np = np.array(X_test_1c)
X_test_1c_np.shape

(567, 5)

In [ ]:
y_tr = train['requester_received_pizza']
y_train = [int(val == True) for val in y_tr]

y_te = test['requester_received_pizza']
y_test = [int(val == True) for val in y_te]


In [ ]:
from sklearn import svm
from sklearn import linear_model


In [ ]:
y_train

In [ ]:
#from sklearn import svm
#from sklearn import linear_model

svm3 = linear_model.SGDClassifier(loss='hinge', penalty='l2', alpha=.001).fit(X_train_1c_np, y_train)
#clf = sklearn.linear_model.SGDClassifier(loss="hinge").fit(X_1c, y_train)

In [ ]:
import sklearn.metrics


test_set_preds = svm3.predict(X_test_1c_np)

accuracy_1a = svm3.score(X_test_1c, y_test)
precision_1a = sklearn.metrics.precision_score(y_test, test_set_preds)
recall_1a = sklearn.metrics.recall_score(y_test, test_set_preds)
f1_1a = sklearn.metrics.f1_score(y_test, test_set_preds)
auc_1a= sklearn.metrics.roc_auc_score(y_test, test_set_preds)

tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, test_set_preds).ravel()
specificity_1a = tn / (tn+fp)

print("Answers for 1c: ")
print("accuracy: " + str(accuracy_1a))
print("precision: " +str(precision_1a))
print("recall: " + str(recall_1a))
print("f1: " + str(f1_1a))
print("auc: " + str(auc_1a))
print(tn)
print(fp)
print(tp)
print(fn)

Answers for 1c: 
accuracy: 0.7372134038800705
precision: 0.0
recall: 0.0
f1: 0.0
auc: 0.5
418
0
0
149


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_set_preds

In [ ]:
y_test

In [ ]:

# 01                    HarmVirtue
# 02                    HarmVice
# 03                    FairnessVirtue
# 04                    FairnessVice
# 05                    IngroupVirtue
# 06                    IngroupVice
# 07                    AuthorityVirtue
# 08                    AuthorityVice
# 09                    PurityVirtue
# 10                    PurityVice
# 11                    MoralityGeneral

Model 4


In [ ]:
d = {1:[], 2:[], 3:[], 4:[], 5:[],6:[],7:[],8:[],9:[],10:[],11:[] }


In [ ]:
d = {1:[], 2:[], 3:[], 4:[], 5:[],6:[],7:[],8:[],9:[],10:[],11:[] }

with open("MoralFoundations_dic_only.txt") as f:
    for line in f:
      #print(line)
      lst = line.split()
      if len(lst) >1:
        #print(lst)
        word = lst[0]

        word = word.replace('*', '')

        indecies = lst[1:]
        for i in indecies:
          toint = int(i)
          d[toint].append(word)

In [ ]:

X_train_1d = []
for text in train['request_text'].tolist():
  total_word_count = len(text.split())

  row =[]
  for i in range(len(d)):
    curr_set = d[i+1]
    set_word_count = 0
    for word in curr_set:
      if word in text:
        set_word_count +=1
    if total_word_count > 0:
      row.append(set_word_count/total_word_count)
    else:
      row.append(0)
  X_train_1d.append(row)

X_train_1d

In [ ]:

X_test_1d = []
for text in test['request_text'].tolist():
  total_word_count = len(text.split())

  row =[]
  for i in range(len(d)):
    curr_set = d[i+1]
    set_word_count = 0
    for word in curr_set:
      if word in text:
        set_word_count +=1
    if total_word_count > 0:
      row.append(set_word_count/total_word_count)
    else:
      row.append(0)
  X_test_1d.append(row)


In [ ]:
X_train_1d_np = np.array(X_train_1d)
X_train_1d_np.shape

(5104, 11)

In [ ]:
X_test_1d_np = np.array(X_test_1d)
X_test_1d_np.shape

(567, 11)

In [ ]:
y_train

In [ ]:
#svm4 = linear_model.SGDClassifier(loss='hinge', penalty='elasticnet')
#svm4.fit(X_train_1d_np, y_train)
from sklearn import svm
from sklearn.linear_model import SGDClassifier

#svm4 = SGDClassifier(loss="hinge", n_iter_no_change=10000).fit(X_train_1d_np, np.array(y_train))
svm4 = svm.SVC(kernel='linear')
svm4.fit(X_train_1d_np, np.array(y_train))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:

import sklearn.metrics


test_set_preds = svm4.predict(X_test_1d_np)

accuracy_1a = svm4.score(X_test_1d_np, y_test)
precision_1a = sklearn.metrics.precision_score(y_test, test_set_preds)
recall_1a = sklearn.metrics.recall_score(y_test, test_set_preds)
f1_1a = sklearn.metrics.f1_score(y_test, test_set_preds)
auc_1a= sklearn.metrics.roc_auc_score(y_test, test_set_preds)

tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, test_set_preds).ravel()
specificity_1a = tn / (tn+fp)

print("Answers for 1d: ")
print("accuracy: " + str(accuracy_1a))
print("precision: " +str(precision_1a))
print("recall: " + str(recall_1a))
print("f1: " + str(f1_1a))
print("auc: " + str(auc_1a))
print(tn)
print(fp)
print(tp)
print(fn)

Answers for 1d: 
accuracy: 0.7372134038800705
precision: 0.0
recall: 0.0
f1: 0.0
auc: 0.5
418
0
0
149


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_set_preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(specificity_1a)

1.0
